# 事前準備
- カタログ、スキーマの作成
  - 編集 > 置き換え > [ytsuchiya -> {ご自身の名前}]
- 検証用データの作成
- サーバレスSQLウェアハウス（Large）の作成

In [0]:
CREATE CATALOG IF NOT EXISTS ytsuchiya;
use catalog `ytsuchiya`;

In [0]:
CREATE SCHEMA IF NOT EXISTS ytsuchiya.tpcds_sf1000;
use schema `tpcds_sf1000`;

In [0]:
CREATE TABLE ytsuchiya.tpcds_sf1000.customer DEEP CLONE samples.tpcds_sf1000.customer;
CREATE TABLE ytsuchiya.tpcds_sf1000.catalog_sales DEEP CLONE samples.tpcds_sf1000.catalog_sales;
CREATE TABLE ytsuchiya.tpcds_sf1000.date_dim DEEP CLONE samples.tpcds_sf1000.date_dim;
CREATE TABLE ytsuchiya.tpcds_sf1000.web_returns DEEP CLONE samples.tpcds_sf1000.web_returns;
CREATE TABLE ytsuchiya.tpcds_sf1000.store_sales DEEP CLONE samples.tpcds_sf1000.store_sales;

In [0]:
DESCRIBE TABLE ytsuchiya.tpcds_sf1000.customer;
DESCRIBE TABLE ytsuchiya.tpcds_sf1000.catalog_sales;
DESCRIBE TABLE ytsuchiya.tpcds_sf1000.date_dim;
DESCRIBE TABLE ytsuchiya.tpcds_sf1000.web_returns;
DESCRIBE TABLE ytsuchiya.tpcds_sf1000.store_sales;

# SQLクエリ最適化ワークショップ
本ワークショップではクエリプロファイルを使用したクエリの実行状況の分析と最適化手法をウォークスルーしながら、実践的なServerlessSQLのパフォーマンスチューニング方法を学びます。<br>
(公式ドキュメント)<br>
https://docs.databricks.com/aws/ja/sql/user/queries/query-profile<br>
(参考記事)<br>
https://qiita.com/Mitsuhiro_itagaki/items/8f8334797e11a4aafd53<br>


<br>前提条件：<br>
- ~~事前にTPC-DSのデータセットが作成されていること~~
  - samplesにtpc-dsデータセットがデフォルトで提供されるようになったため不要（[Doc](https://docs.databricks.com/aws/ja/sql/tpcds-eval)）
- ~~ユーザはServelessSQL(L)サイズに対するCan Moniotor権限以上を付与されていること (Spark UIを使用するため)~~
  - Spark UI を使用するためには、サーバレスでないSQLウェアハウスの使用が必要

#### ワークショップ環境に接続

In [0]:
%sql
-- ワークショップではリザルトキャッシュの利用を必ずOFFにしておきます
set use_cached_result = false;

-- 環境確認のためのクエリ
select current_catalog() as env
union all
select current_database();

#### ワークテーブルの作成
##### detail_XXXのテーブル名称を適宜修正してください。<br>

In [0]:
select * from ytsuchiya.tpcds_sf1000.customer;

In [0]:
-- テスト用のマスターテーブル
-- テーブル名称を適宜修正してください。
create or replace table master_XXX as  
select 
distinct c_current_hdemo_sk as ID,
'Skew test' as val
from customer where c_current_hdemo_sk between 0 and 6;

select * from master_XXX order by id;

In [0]:
-- テスト用の明細テーブル
-- CSVテキスト換算で約300GB (14億レコード)
-- テーブル名称を適宜修正してください。
-- ServerlessSQL(L)で3分程度
CREATE OR REPLACE TABLE `detail_XXX` 
as 
select 
case 
  when cs_bill_hdemo_sk between 1    and  500 then 1 
  when cs_bill_hdemo_sk between  501 and 1000 then 2
  when cs_bill_hdemo_sk between 1001 and 1500 then 3
  when cs_bill_hdemo_sk between 1501 and 2000 then 4
  when cs_bill_hdemo_sk between 2001 and 2500 then 5
  when cs_bill_hdemo_sk >= 2501 then 6
else 0 end AS ID 
,* from catalog_sales 
order by cs_net_profit;

select * from detail_XXX ;

##　クエリ1:Liqud Clusteringによるクエリプランの最適化
https://docs.databricks.com/aws/ja/delta/clustering

#### チューニング前クエリ
実行後にクエリプロファイラを確認します

In [0]:
%sql
-- ワークショップではリザルトキャッシュの利用を必ずOFFにしておきます
set use_cached_result = false;

select
  cs_bill_customer_sk,
  cs_catalog_page_sk,
  sum(cs_list_price) as v01,
  sum(cs_ext_sales_price) as v02,
  sum(cs_wholesale_cost) as v03,
  sum(cs_ext_discount_amt) as v04,
  sum(cs_ext_sales_price) as v05,
  sum(cs_ext_wholesale_cost) as v06,
  sum(cs_ext_list_price) as v07,
  sum(cs_ext_tax) as v08,
  sum(cs_coupon_amt) as v09,
  sum(cs_ext_ship_cost) as v10,
  sum(cs_net_paid) as v11,
  sum(cs_net_paid_inc_tax) as v12,
  sum(cs_net_paid_inc_ship) as v13,
  sum(cs_net_paid_inc_ship_tax) as v14,
  sum(cs_net_profit) as v15
from
  detail_XXX
where
  cs_sold_date_sk = 2451659
group by
  cs_bill_customer_sk,
  cs_catalog_page_sk
order by
  v01 desc
limit 10;

#### クエリプロファイルの確認
<br>**(1)SQL処理全体のメトリクスを確認**<br>
- フィルターレートが0%なのでWhere条件が付与されているもののテーブルがフルスキャンになっているようです<br>
<img src='https://qiita-image-store.s3.ap-northeast-1.amazonaws.com/0/1346521/f6f488ec-b2ff-43aa-bb5d-ef6cea0b68f5.png' width='400'/>

<br><br>**(2)トップオペレーターでボトルネックを確認**<br>
- 最も時間のかかっている処理はScan Tableです<br>
<img src='https://qiita-image-store.s3.ap-northeast-1.amazonaws.com/0/1346521/92388c69-b86b-4c42-be7f-6fbbe45d532f.png' width='400'/>

<br><br>**(3)該当テーブルアクセスのメトリクスを確認**<br>
- フィルタ(pruned)のメトリクスはFile/Sizeとも0となっている<br>
<img src='https://qiita-image-store.s3.ap-northeast-1.amazonaws.com/0/1346521/a3353fdc-2ca2-4e87-9e2f-c0d0553f208a.png' width='400'/>

<br><br>**(4)テーブルアクセスのフィルタ条件(Filters)を確認**<br>
- フィルタ条件cs_sold_dateカラムのみ<br>
<img src='https://qiita-image-store.s3.ap-northeast-1.amazonaws.com/0/1346521/9627fc5b-1bae-49d4-aacd-feafe68a9a45.png' width='400'/>

####　リキッドクラスタリングの設定
フィルタ条件に使用されるカラムをクラスタリングキーに設定することで、データの物理配置を最適化し、I/Oの削減効率を高めます。

<img src='https://qiita-image-store.s3.ap-northeast-1.amazonaws.com/0/1346521/e94e968f-ebdb-4eb6-a460-2a45bba374b9.png' width='800'/>

In [0]:
-- 最大4カラムまで指定可能
ALTER TABLE `detail_XXX` CLUSTER BY (cs_sold_date_sk);

#### Optimizeコマンドによる物理再配置
ServerlessSQL(L)で3分程度</br>

In [0]:
OPTIMIZE detail_XXX FULL;

#### 統計情報の取得<br>
予測最適化が有効になっている場合はデータ更新時に統計情報が自動で取得されtいます<br><br>

In [0]:
ANALYZE TABLE detail_XXX COMPUTE STATISTICS FOR ALL COLUMNS;

#### テーブル情報の確認

In [0]:
DESCRIBE EXTENDED detail_XXX

#### 再度同じクエリを実行してみます
I/Oボトルネックが解消されたかメトリクスを確認します

In [0]:
%sql
-- ワークショップではリザルトキャッシュの利用を必ずOFFにしておきます
set use_cached_result = false;

select
  cs_bill_customer_sk,
  cs_catalog_page_sk,
  sum(cs_list_price) as v01,
  sum(cs_ext_sales_price) as v02,
  sum(cs_wholesale_cost) as v03,
  sum(cs_ext_discount_amt) as v04,
  sum(cs_ext_sales_price) as v05,
  sum(cs_ext_wholesale_cost) as v06,
  sum(cs_ext_list_price) as v07,
  sum(cs_ext_tax) as v08,
  sum(cs_coupon_amt) as v09,
  sum(cs_ext_ship_cost) as v10,
  sum(cs_net_paid) as v11,
  sum(cs_net_paid_inc_tax) as v12,
  sum(cs_net_paid_inc_ship) as v13,
  sum(cs_net_paid_inc_ship_tax) as v14,
  sum(cs_net_profit) as v15
from
  `detail_XXX`
where
  cs_sold_date_sk = '2451659'
group by
  cs_bill_customer_sk,
  cs_catalog_page_sk
order by
  v01 desc
limit 100;

#### Liquid Clustering設定後のクエリプロファイルを確認
<img src='https://qiita-image-store.s3.ap-northeast-1.amazonaws.com/0/1346521/5fdc3d1d-ce5a-4f59-897d-b1edf6bfd70b.png' width='1000'/>

#### 補足：予測最適化(Predictive Optimization)
https://docs.databricks.com/aws/ja/optimizations/predictive-optimization

<img src='https://qiita-image-store.s3.ap-northeast-1.amazonaws.com/0/1346521/68210a6f-dd0b-4006-952b-22768e60508d.png' width='1000'/>

## クエリ2:Photonエンジンの有効利用
https://docs.databricks.com/aws/ja/compute/photon

#### Photon - コンピューティング処理の高速化
<img src='https://qiita-image-store.s3.ap-northeast-1.amazonaws.com/0/1346521/bde7852f-cb9c-408e-a183-5172a721551e.png' width='800'/>

#### Photon未対応クエリの実行
2025年7月時点でPhoton未対応のPIVOT関数を使用してメトリクスを確認

In [0]:
%sql
-- ワークショップではリザルトキャッシュの利用を必ずOFFにしておきます
set use_cached_result = false;

SELECT
  c_birth_year,
  COUNT(JAPAN) AS JAPAN,
  COUNT(ANGUILLA) AS ANGUILLA,
  COUNT(ANTARCTICA) AS ANTARCTICA,
  COUNT(ARUBA) AS ARUBA,
  COUNT(ARMENIA) AS ARMENIA,
  COUNT(FINLAND) AS FINLAND,
  COUNT(UKRAINE) AS UKRAINE,
  COUNT(URUGUAY) AS URUGUAY
FROM
  customer
    PIVOT (count(c_birth_country) FOR c_birth_country IN ('JAPAN','ANGUILLA','ANTARCTICA','ARUBA','ARMENIA','FINLAND','UKRAINE','URUGUAY'))
WHERE c_birth_year is not null
GROUP BY
  c_birth_year
ORDER BY
  1;

#### クエリプロファイル + SparkUIの確認
<br>**(1)トップオペレーターでボトルネックを確認**<br>
- トップオペレーターでSparkエンジンへの切り替え(Columner To Row)のオーバヘッドを確認<br>
<img src='https://qiita-image-store.s3.ap-northeast-1.amazonaws.com/0/1346521/7957abcf-5d75-4528-a861-a8561988393b.png' width='300'/>

<br><br>**(2)SparkUIを起動(SQLウェアハウスのCanMamage権限が必要、ServerlessではSparkUIの確認不可)**<br>
- Photon利用率を確認したら、SparkUIを起動してPhotonエンジンが利用できない原因を特定します。
<img src='https://qiita-image-store.s3.ap-northeast-1.amazonaws.com/0/1346521/72dfe408-74dc-4008-89df-bcb363177df1.png' width='500'/>

- SparkUIの下部にあるDetailsを確認
<img src='https://qiita-image-store.s3.ap-northeast-1.amazonaws.com/0/1346521/d001c4df-d0a3-4d46-a6e4-37dafd5c8f85.png' width='500'/>

- Detailsの最後に出力されている情報からPhotonエンジンが利用できない要因を確認
<img src='https://qiita-image-store.s3.ap-northeast-1.amazonaws.com/0/1346521/5ea91628-ef48-4d29-b3d6-991d4899441d.png' width='1200'/>

In [0]:
-- 
-- EPLAINコマンドでもPhoton Explanationの情報を出力できます
-- 

EXPLAIN FORMATTED
SELECT
  c_birth_year,
  COUNT(JAPAN) AS JAPAN,
  COUNT(ANGUILLA) AS ANGUILLA,
  COUNT(ANTARCTICA) AS ANTARCTICA,
  COUNT(ARUBA) AS ARUBA,
  COUNT(ARMENIA) AS ARMENIA,
  COUNT(FINLAND) AS FINLAND,
  COUNT(UKRAINE) AS UKRAINE,
  COUNT(URUGUAY) AS URUGUAY
FROM
  customer
    PIVOT (count(c_birth_country) FOR c_birth_country IN ('JAPAN','ANGUILLA','ANTARCTICA','ARUBA','ARMENIA','FINLAND','UKRAINE','URUGUAY'))
WHERE c_birth_year is not null
GROUP BY
  c_birth_year
ORDER BY
  1;

#### Photon対応クエリへ書き換え
- 原因を特定したらPhoton対応のクエリに変換します

##### アシスタントを使用してクエリを書き換えてみましょう！

以下のようにPIVOT構文を使わず、各国ごとにCASE式でカウントすることでPhotonエンジン対応となります：


#### Photon対応クエリの実行

In [0]:
%sql
-- ワークショップではリザルトキャッシュの利用を必ずOFFにしておきます
set use_cached_result = false;

SELECT
  c_birth_year,
  SUM(CASE WHEN c_birth_country = 'JAPAN' THEN 1 ELSE 0 END) AS JAPAN,
  SUM(CASE WHEN c_birth_country = 'ANGUILLA' THEN 1 ELSE 0 END) AS ANGUILLA,
  SUM(CASE WHEN c_birth_country = 'ANTARCTICA' THEN 1 ELSE 0 END) AS ANTARCTICA,
  SUM(CASE WHEN c_birth_country = 'ARUBA' THEN 1 ELSE 0 END) AS ARUBA,
  SUM(CASE WHEN c_birth_country = 'ARMENIA' THEN 1 ELSE 0 END) AS ARMENIA,
  SUM(CASE WHEN c_birth_country = 'FINLAND' THEN 1 ELSE 0 END) AS FINLAND,
  SUM(CASE WHEN c_birth_country = 'UKRAINE' THEN 1 ELSE 0 END) AS UKRAINE,
  SUM(CASE WHEN c_birth_country = 'URUGUAY' THEN 1 ELSE 0 END) AS URUGUAY
FROM
  customer
WHERE
  c_birth_year IS NOT NULL
GROUP BY
  c_birth_year
ORDER BY
  c_birth_year;

##### 実行後にプランを確認してください

<img src='https://qiita-image-store.s3.ap-northeast-1.amazonaws.com/0/1346521/36e89d87-41d8-4637-9c1d-9e8e9e9e8987.png' width='400'/>

##　クエリ3:ユーザ定義関数の使用時の注意
https://docs.databricks.com/ja/udf/unity-catalog.html</br>


#### LANGUAGE PYTHONで作成したユーザ定義関数

In [0]:
-- うるう年を判定するPython関数
CREATE OR REPLACE FUNCTION isleapyear_py_ita(year INT)
RETURNS BOOLEAN
LANGUAGE PYTHON  -- Pythonの関数はPythonエンジンとSQLエンジン間でデータ通信が発生します
DETERMINISTIC  -- クエリオプティマイザは関数が同じ入力に対して常に同じ結果を返すことを前提に最適化を行うことができます
AS $$
import calendar
return calendar.isleap(year) if year else None
$$;

In [0]:
%sql
-- ワークショップではリザルトキャッシュの利用を必ずOFFにしておきます
set use_cached_result = false;

-- LANGUAGE PYTHONで作成した関数を使用したクエリ
select
    d_year,
    isleapyear_py_ita(d_year) as d_year_leap,
    sum(wr_return_amt) as wr_return_amt_total
  from
    web_returns,
    date_dim
  where
    wr_returned_date_sk = d_date_sk
    and d_year in (2000,2001,2002,2003)
  group by 1, 2
  order by 1

#### SQLプラン確認: クエリプロファイル
<br>**トップオペレーターでボトルネックを確認**<br>
- UDFの実行(Scalar UDF)で時間がかかっていることが確認できます<br>
- Pythonユーザ定義関数の実行はPythonエンジンとSQLエンジン間でのプロセス間通信が発生することが遅延の原因です<br>
<img src='https://qiita-image-store.s3.ap-northeast-1.amazonaws.com/0/1346521/884f3a39-86c7-4cca-804a-06a13edcab81.png' width='300'/>

#### LANGUAGE SQLで作成したユーザ定義関数

In [0]:
-- うるう年を判定するSQL関数
CREATE OR REPLACE FUNCTION isleapyear_sql(year INT)
RETURNS BOOLEAN
LANGUAGE SQL   -- SQL関数はSQLエンジン内で処理されます
DETERMINISTIC  -- クエリオプティマイザは関数が同じ入力に対して常に同じ結果を返すことを前提に最適化を行うことができます
RETURN CASE
    WHEN year % 400 = 0 THEN TRUE
    WHEN year % 100 = 0 THEN FALSE
    WHEN year % 4 = 0 THEN TRUE
    ELSE FALSE
END;

In [0]:
%sql
-- ワークショップではリザルトキャッシュの利用を必ずOFFにしておきます
set use_cached_result = false;

-- LANGUAGE SQLで作成した関数を使用したクエリ
select
    d_year,
    isleapyear_sql(d_year) as d_year_leap,
    sum(wr_return_amt) as wr_return_amt_total
  from
    web_returns,
    date_dim
  where
    wr_returned_date_sk = d_date_sk
  and d_year in (2000,2001,2002,2003)
  group by 1, 2
  order by 1

##### 実行後にプランを確認してください>
<img src='https://qiita-image-store.s3.ap-northeast-1.amazonaws.com/0/1346521/25897ca0-7bab-42ad-bbf9-859134941d69.png' width='400'/>

##　クエリ4:結合プランの調整
https://docs.databricks.com/aws/ja/sql/language-manual/sql-ref-syntax-qry-select-hints</br>

#### ソートマージ結合の例
Sort Merge Join が選ばれている場合はPhotonエンジンが利用できないため改善の余地があることが多く、統計情報の最新化やテーブルサイズに応じたプランの見直しが効果的です

In [0]:
%sql
-- ワークショップではリザルトキャッシュの利用を必ずOFFにしておきます
set use_cached_result = false;

-- store_salesとdate_dimテーブルを使用したソートマージ結合クエリ
-- 便宜上、明示的にMERGEヒントを使用して再現しています。

SELECT /*+ MERGE(dd) */
  ss_item_sk,
  ss_ticket_number,
  d_date,
  ss_sales_price
FROM
  tpcds_sf1000.store_sales ss
JOIN
  tpcds_sf1000.date_dim dd
ON
  ss.ss_sold_date_sk = dd.d_date_sk
WHERE
  dd.d_year between 1990 and 2000
ORDER BY
  ss_sales_price
Limit 100;

##### 実行後にプランを確認してください>
- ソートマージ結合ではPhotonエンジンが利用できないため処理の遅延が発生しやすくなります。

<img src='https://qiita-image-store.s3.ap-northeast-1.amazonaws.com/0/1346521/b9f604f9-cf5d-45d3-b4fb-b11003741dbd.png' width='1200'/>

#### ブロードキャスト結合の例
ヒントにより静的に計画されたブロードキャスト結合は、AQE によって動的に計画された結合よりも一般的にパフォーマンスは優れています。<br>
これは結合対象のテーブルをスキャンし 、実際のサイズが取得されるまで AQEが実行プランをブロードキャスト結合に切り替えない場合があるためです。<br> 
したがって、クエリをよく理解していれば、ブロードキャストヒントを使用することは依然として良い選択です。

In [0]:
%sql
-- ワークショップではリザルトキャッシュの利用を必ずOFFにしておきます
set use_cached_result = false;

-- store_salesとdate_dimテーブルを使用したブロードキャスト結合クエリ
-- 明示的にBROADCASTヒントを使用して再現しています。ブロードキャストされるテーブルを指定します。
SELECT /*+ BROADCAST(dd) */
  ss_item_sk,
  ss_ticket_number,
  d_date,
  ss_sales_price
FROM
  tpcds_sf1000.store_sales ss
JOIN
  tpcds_sf1000.date_dim dd
ON
  ss.ss_sold_date_sk = dd.d_date_sk
WHERE
  dd.d_year between 1990 and 2000
ORDER BY
  ss_sales_price
LIMIT 100;

##### 実行後にプランを確認してください
<img src='https://qiita-image-store.s3.ap-northeast-1.amazonaws.com/0/1346521/792a7ffa-e0b3-40ab-b914-3ab00858e320.png' width='1200'/>

#### シャッフルハッシュ結合の例
SHUFFLE_HASHヒントは結合対象が両方とも大きなデータセットの場合に有効な結合プランです。

In [0]:
%sql
-- ワークショップではリザルトキャッシュの利用を必ずOFFにしておきます
set use_cached_result = false;

-- ヒントではメモリ上にハッシュされるテーブルを指定します。
SELECT /*+ SHUFFLE_HASH(dd) */
  ss_item_sk,
  ss_ticket_number,
  d_date,
  ss_sales_price
FROM
  tpcds_sf1000.store_sales ss
JOIN
  tpcds_sf1000.date_dim dd
ON
  ss.ss_sold_date_sk = dd.d_date_sk
WHERE
  dd.d_year between 1990 and 2000
ORDER BY
  ss_sales_price
LIMIT 100;

##### 実行後にプランを確認してください

##　クエリ5:  シャッフル結合時のスキューの改善

AQE(Adaptive Query Execution) はデフォルトで有効になっています。 以下の主要な機能があります。

| 機能                                                | 説明                                                                | 典型的な効果                           |
| ------------------------------------------------- | ----------------------------------------------------------------- | -------------------------------- |
| 1. **Join戦略の自動切り替え**                              | 実行時に `BroadcastHashJoin`・`ShuffleHashJoin`・`SortMergeJoin` を動的に選択 | テーブルサイズやデータ分布に応じて最適なJoin方式に変更される |
| 2. **Skew Join 最適化（Skew Join Split）**             | シャッフルされたパーティションに偏りがあると、重いパーティションだけ分割して並列化                         | タスクスキュー（Straggler）によるジョブ遅延を防止    |
| 3. **動的再パーティショニング（Coalesced Shuffle Partitions）** | 実行中にシャッフル出力のパーティション数を調整（小さいタスクは結合）                                | 不要に細かいパーティションを削減 → タスクオーバーヘッドを低減 |
| 4. **Exchangeノードの統合（Exchange reuse）**             | 同じシャッフルを複数回使う場合に結果を共有                                       | コスト削減・パフォーマンス向上（特にCTEや複雑なクエリ）    |
| 5. **空パーティションの除外（Empty Partition Pruning）**       | 実行時に読み込む必要がないパーティションをスキップ                                         | I/O 削減（例：`JOIN` に絡むテーブルの片方が空）    |
| 6. **動的最適化のログ出力**                                 | 実行プランに `Adaptive Plan` として反映、Query Profile にも詳細記録                 | デバッグ・性能チューニングに役立つ                |


#### Skew Join 最適化の例
シャッフルされたパーティションの偏りを検知すると重いパーティションを検出後に再分割して並列化する最適化された実行プランを自動生成します。

In [0]:
%sql
-- ワークショップではリザルトキャッシュの利用を必ずOFFにしておきます
set use_cached_result = false;

SELECT /*+ SHUFFLE_HASH(T2) */
  T1.ID, 
  T2.val,
  T1.cs_net_paid
FROM detail_XXX as T1
JOIN master_XXX as T2
  ON T1.ID = T2.ID
ORDER BY 2 desc
limit 10

##### 実行後にプランを確認してください
<img src='https://qiita-image-store.s3.ap-northeast-1.amazonaws.com/0/1346521/9adea6a4-1c0d-4b4a-935f-2eee910745d5.png' width='1200'/>

#### Skew Join 最適化が機能しないクエリの例
一部のプランではSkewが自動検出されないケースが報告されています（将来的に修正予定）<br>
このようなケースではクエリ実行プランを変更するか、クエリを書き換えで対応しなければならないケースもあります。

In [0]:
%sql
-- ワークショップではリザルトキャッシュの利用を必ずOFFにしておきます
set use_cached_result = false;

SELECT /*+ SHUFFLE_HASH(T2) */
  T1.ID, 
  AVG(T1.cs_net_paid) as avg_cs_net_paid,
  COUNT(distinct T2.val) as count_val 
FROM detail_XXX as T1
JOIN master_XXX as T2
  ON T1.ID = T2.ID
GROUP BY T1.ID
ORDER BY T1.ID


##### 実行後にプランを確認してください
<img src='https://qiita-image-store.s3.ap-northeast-1.amazonaws.com/0/1346521/6efb5385-b0ed-49c7-b8e7-24b4d7bd2152.png' width='1200'/>

- Spark UIでの確認<br>
**ShuffuleReadのデータ容量は各タスクで128MB程度になるように調整すると良い。なるべく16MB〜256MBに収まるように！！**

<img src='https://qiita-image-store.s3.ap-northeast-1.amazonaws.com/0/1346521/077d1cfc-147f-458e-8f0c-8873946eaadd.png' width='1200'/>

#### 回避方法1:ブロードキャスト結合の使用（ブロードキャスト結合が可能であれば）

In [0]:
%sql
-- ワークショップではリザルトキャッシュの利用を必ずOFFにしておきます
set use_cached_result = false;

SELECT /*+ BROADCAST(T2) */
  T1.ID, 
  AVG(T1.cs_net_paid) as avg_cs_net_paid,
  COUNT(distinct T2.val) as count_val 
FROM detail_XXX as T1
JOIN master_XXX as T2
  ON T1.ID = T2.ID
GROUP BY T1.ID
ORDER BY T1.ID

#### 回避方法2:事前集計によりシャッフル対象のレコード数を削減する
COUNT(DISTINCT) をJOIN後にやると高コストなので、先に T2 側で ID ごとに val の種類数を数えるクエリに変更する

In [0]:

%sql

WITH t2_agg AS (
  SELECT ID, COUNT(DISTINCT val) AS count_val
  FROM master_XXX
  GROUP BY ID
)

SELECT /*+ SHUFFLE_HASH(t2_agg) */
  T1.ID, 
  AVG(T1.cs_net_paid) AS avg_cs_net_paid,
  t2_agg.count_val
FROM detail_XXX AS T1
JOIN t2_agg
  ON T1.ID = t2_agg.ID
GROUP BY T1.ID, t2_agg.count_val
ORDER BY T1.ID